# Alcance del Notebook
Queremos replicar el expermiento de Frank Rosenblatt, para eso, vamos a cargar un dataset con 1000 imagenes en blanco y negro en las cuales tenemos distinguidas y etiquetadas aquellas que tienen cuadrados a la izquierda y a la derecha.

In [ ]:
import numpy as np
import requests
import os
import zipfile

import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt

## Carga del dataset
Tomamos un dataset previamente creado con 1000 imágenes de 38x38 con cuadrados desde 1x1 hasta 11x11 que se encuentran completamente a derecha o izquierda. Por cada tamaño de cuadrado tenemos 10 de cada tipo.

In [ ]:
tfrecord_zip = "https://drive.google.com/u/0/uc?id=1AtgJa4sE0PeI-ooy5eBvSETxr1kIYt0A&export=download"

dest_path = os.path.join(os.path.curdir, 'data')
file_name = 'dataset.zip'
file_path = os.path.join(os.path.curdir, file_name)

response = requests.get(tfrecord_zip)
with open(file_path, 'wb') as dest_file:
    dest_file.write(response.content)

# Descomprimimos el archivo ZIP
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(dest_path)

# Cargamos el dataset desde el archivo TFRecord para despues poder manipularlo con TensorFlow
dataset = tf.data.Dataset.load('/content/rosenblatt_sample.tfrecord')
# Mezclamos las imagenes de forma azarosa, pero con un seed para que siempre sea el mismo azar y podamos hacer pruebas determinísticas
dataset = dataset.shuffle(1000, seed=17, reshuffle_each_iteration=False)

## Dibujamos las imágenes del Dataset (algunas)

In [ ]:
# Definir el tamaño de la cuadrícula de subplots
rows = 20
columns = 5

# Crear una figura y subplots
fig, axs = plt.subplots(rows, columns, figsize=(10, 30))

# Iterar a través de las imágenes y subplots
row = 0
column = 0
for image, label in dataset.take(100):  # Agarramos solo 100
  axs[row, column].imshow(image, cmap=mpl.cm.binary)
  axs[row, column].set_title(f"Etiqueta {int(label[0])}")
  axs[row, column].axis('off')  # Desactivar los ejes
  if (column+1) % 5 == 0:
    column = 0
    row += 1
  else:
    column += 1

plt.tight_layout()
plt.show()

## Preparación de los datos
Vamos a querer dividir la totalidad de muestras en subconjuntos de entrenamiento y subconjuntos de prueba

In [ ]:
train_dataset = dataset.take(800)
test_dataset = dataset.skip(800).take(200)

## Normalización
Al trabajar con imagenes una práctica comun es representar la imagen como un vector de una sola dimensión, entonces en lugar de una matriz de 38x38 vamos a contar con vectores de 1444 valores. Esto es útil para realizar operaciones de manera mas sencilla e intuitiva. Otra acción que tomamos y también es práctica habitual es representar los colores en escalas entre 1 y 0.

In [ ]:
def normalize(image, label):
    return tf.cast(image, tf.float32) / 255., label

# Transforma la imagen un único vector fila de 784 elementos
def flatten(image, label):
    image = tf.reshape(image, shape=[1, -1])
    return image, label

def preprocess_pipeline(dataset_to_process):
    dataset_to_process = dataset_to_process.map(normalize)
    dataset_to_process = dataset_to_process.map(flatten)
    return dataset_to_process

train_dataset = preprocess_pipeline(train_dataset)
test_dataset = preprocess_pipeline(test_dataset)

## Ahora si, a entrenar!
Habitualmente se setea una cantidad de "epocas" en las que un modelo debe entrenar de forma iterativa.

Para este problema particular, nos interesa ir moviendonos por la gradiente a medida que nuestra neurona va arrojando resultados para cada entrada. Si bien esto
parece complejo, herramientas como TensorFlow vienen a salvarnos para no tener que pensar tan fuertemente en la parte matemática del asunto.

Como parte del algoritmo, tenemos que ver cuales son nuestras funciones de activación y de pérdida. Nuevamente para este problema se conoce que las funciones son:

Funcion Sigmoide (Activación)
$$\sigma(x) = \frac{1}{1 + e^{-x}}$$

Binary Cross Entropy (Pérdida)
$$ H(true\_val, predict) = - (true\_val \log(predict) + (1 - true\_val) \log(1 - predict))$$

No queda mas que programar el algoritmo:

In [ ]:
# Nuevamente tomamos un random seedeado para poder replicar el mismo entrenamiento en futuras interacciones
tf.random.set_seed(234)

w = tf.zeros(shape=[1444,1])
b = tf.zeros(shape=[])
w = tf.Variable(w)
b = tf.Variable(b)

model_variables = [w, b]

epochs = 25  # Seteamos las épocas para "entrenar"
learning_rate = 0.005
# Data de output:
train_losses = []
train_accuracy = []
test_losses = []

for epoch in range(epochs):
    epoch_accuracies = []
    epoch_losses = []
    epoch_test_losses = []
    test_accuracies = []

    with tf.GradientTape() as tape:
        # Iteración sobre los conjuntos de entrenamiento
        for image, true_label in train_dataset.take(800):
            # Primero obtenemos del dataset el valor que quermos predecir,
            # o sea, la etiqueta que indica si es derecha o izquierda.
            # Lo casteamos a float ya que lo tenemos almacenado como un entero
            # dentro de un arreglo de dimension 1.
            true_label = tf.cast(true_label[0], tf.float32)
            # Aplicamos la ecuacion 'y = w*x + b', pero en vectores:
            # logits = w*image + b
            logits = tf.add(tf.matmul(image, w), b)
            # Dentro de logits podemos tener cualquier valor flotante y ademas
            # la operacion realizada con tf retorna este valor como un arreglo de arreglos.
            # Por eso usamos squeeze que solamente le quita una dimension.
            logits = tf.squeeze(logits)
            # aplicamos sigmoide a logits para tenerlo entre 0 y 1 y calculamos
            # la perdida respecto de true_label usando binary_cross_entropy
            loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=true_label, logits=logits)
            epoch_losses.append(loss)

            # Calculamos el accuracy de nuestra prediccion y guardamos
            prediction = 1.0 if tf.math.sigmoid(logits)>0.5 else 0.0
            accuracy = 1.0 if prediction == true_label else 0.0
            epoch_accuracies.append(accuracy)

        epoch_accuracies_mean = tf.reduce_mean(epoch_accuracies)
        # Computamos el gradiente de la funcion de perdida a minimizar
        epoch_losses_mean = tf.reduce_mean(epoch_losses)
    # Obtenemos el gradiente para minimizar la funcion de perdida
    grads = tape.gradient(epoch_losses_mean, model_variables)
    print(f"Epoch: {epoch}")
    print(f"Loss: {epoch_losses_mean:.3f}")
    print(f"Accuracy: {epoch_accuracies_mean:.3f}")

    # Actualizamos el modelo moviendo w y b en direccion del gradiente
    # en la cantidad indicada por nuestro learning_rate.
    for dv, variable in zip(grads, model_variables):
        variable.assign_sub(learning_rate * dv)

    train_losses.append(epoch_losses_mean)

    # Iteración sobre los conjuntos de validación
    for image, true_label in test_dataset.take(200):
      true_label = tf.cast(true_label[0], tf.float32)
      logits = tf.add(tf.matmul(image, w), b)
      logits = tf.squeeze(logits)
      loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=true_label, logits=logits)
      epoch_test_losses.append(loss)

      prediction = 1.0 if tf.math.sigmoid(logits)>0.5 else 0.0
      accuracy = 1.0 if prediction == true_label else 0.0
      test_accuracies.append(accuracy)

    test_accuracies_mean = tf.reduce_mean(test_accuracies)

    # keep track of the loss
    test_epoch_losses_mean = tf.reduce_mean(epoch_test_losses)
    test_losses.append(test_epoch_losses_mean)
    print(f"Test loss: {test_epoch_losses_mean:.3f}")
    print(f"Test Accuracy: {test_accuracies_mean:.3f}")
    print("--------------------------------------------------")

# Función que representa al modelo entrenado, dado una imagen de las mismas condiciones, devuelve si el cuadrado está a la derecha o izquierda
def predict(image):
  logits = tf.add(tf.matmul(image, w), b)
  logits = tf.squeeze(logits)
  return "Derecha" if tf.math.sigmoid(logits)>0.5 else "Izquierda"

# Wait... como le creo al modelo?
Adicionalmente, creamos un set de funciones que nos va a permitir crear imagenes con cuadrados de las mismas características, luego le pasamos esta imagen al modelo y lo ponemos a prueba!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Image

# Variable para almacenar la imagen resultante
image_to_predict = None
output_widget = widgets.Output()

def draw_square(pos_x, pos_y, size):
    global image_to_predict
    # Crear una imagen en blanco y negro
    imagen = np.zeros((38, 38), dtype=np.uint8)
    # Dibujar un cuadrado blanco en la posición y tamaño especificados
    imagen[pos_y:pos_y + size, pos_x:pos_x + size] = 1
    # Mostrar la imagen
    plt.imshow(imagen, cmap='gray', vmin=0, vmax=1)
    plt.axis('off')
    plt.show()
    image_to_predict =1.0 - imagen.reshape(1, -1).astype(np.float32)
    plt.close()

# Campos interactivos
pos_x = widgets.IntSlider(min=0, max=38, step=1, value=0, description='pos_x:')
pos_y = widgets.IntSlider(min=0, max=38, step=1, value=0, description='pos_y:')
size = widgets.IntSlider(min=1, max=38, step=1, value=10, description='size:')

# Botón para dibujar el cuadrado
draw_button = widgets.Button(description='Dibujar Cuadrado')

# Función para manejar el clic en el botón
def on_button_click(b):
  draw_square(pos_x.value, pos_y.value, size.value)

# Asociar la función con el evento de clic en el botón
draw_button.on_click(on_button_click)

# Mostrar campos y botón
display(pos_x, pos_y, size, draw_button)

# Mostrar la imagen generada (si existe)
if image_to_predict is not None:
    display(Image(data=image_to_predict))

In [ ]:
# Ahora, que el modelo prediga...
predict(image_to_predict)